<a href="https://colab.research.google.com/github/Junseokee/Deep_Learning/blob/main/220919_%EC%A7%88%EB%AC%B8%EC%97%90_%EB%8B%B5%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 질의 응답 과제
질의 응답(Question Answering) 이란 질문에 답하는 과지입니다. 이 창에서는 LG CNS가 공개한 KorQUAD v.1 데이터셋을 사용


## 7-1 질의응답 모델 훑어보기
  - 질의 응답(Question Answering)이란 질문에 답을 하는 과제
  - 질의 응답 과제의 유형은 다양
  - 다음 과제에서 진행하는 질의 응답 과제는 질문에 대한 답을 지문에서 찾는것
  

In [ ]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 구글드라이브와 연결
from google.colab  import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!mkdir /content/Korpora
!mkdir /content/Korpora/korquad-v1/
!wget -O /content/Korpora/korquad-v1/KorQuAD_v1.0_train.json https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
!wget -O /content/Korpora/korquad-v1/KorQuAD_v1.0_dev.json https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json

mkdir: cannot create directory ‘/content/Korpora’: File exists
mkdir: cannot create directory ‘/content/Korpora/korquad-v1/’: File exists
--2022-09-19 05:47:05--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘/content/Korpora/korquad-v1/KorQuAD_v1.0_train.json’

/content/Korpora/ko 100%[===================>]  36.74M  81.2MB/s    in 0.5s    

2022-09-19 05:47:05 (81.2 MB/s) - ‘/content/Korpora/korquad-v1/KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2022-09-19 05:47:05--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.

In [ ]:
import torch
from ratsnlp.nlpbook.qa import QATrainArguments
args = QATrainArguments(
    pretrained_model_name='beomi/kcbert-base',
    downstream_corpus_name="korquad-v1",
    downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-qa',
    max_seq_length=128,
    max_query_length=32,
    doc_stride=64,
    batch_size=32,
    learning_rate=5e-5,
    epochs=1,
    tpu_cores=0,
    seed=7,
)

### 랜덤시드 고정

In [ ]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


### 로거 설정
- 각종 로그들을 출럭하는 로거를 설정

In [ ]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=4, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_size=32, fp16=False, tpu_cores=0, tqdm_enabled=True)
INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=4, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e

## 토크나이저 준비하기
- 이준범님이 공개하신 kcbert-base 모델이 사용하는 토크나이저를 선언

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False, # 소문자 변환여부 한글이라 False
)

In [ ]:
import json

json.load(open('/content/Korpora/korquad-v1/KorQuAD_v1.0_dev.json',
               'r', encoding='utf-8'))['data'][0]['paragraphs'][0:2]

[{'qas': [{'answers': [{'text': '1989년 2월 15일', 'answer_start': 0}],
    'id': '6548850-0-0',
    'question': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?'},
   {'answers': [{'text': '임수경', 'answer_start': 125}],
    'id': '6548850-0-1',
    'question': '1989년 6월 30일 평양축전에 대표로 파견 된 인물은?'},
   {'answers': [{'text': '1989년', 'answer_start': 0}],
    'id': '6548853-0-0',
    'question': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?'},
   {'answers': [{'text': '학생회관 건물 계단', 'answer_start': 365}],
    'id': '6548853-0-1',
    'question': '임종석을 검거한 장소는 경희대 내 어디인가?'},
   {'answers': [{'text': '서울지방경찰청 공안분실', 'answer_start': 457}],
    'id': '6548853-0-2',
    'question': '임종석이 조사를 받은 뒤 인계된 곳은 어딘가?'},
   {'answers': [{'text': '임종석', 'answer_start': 87}],
    'id': '6332405-0-0',
    'question': '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?'},
   {'answers': [{'text': '여의도 농민 폭력 시위', 'answer_start': 13}],
    'id': '6332405-0-1',
    'question': '임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가?'}],
  '

In [ ]:
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset
corpus = KorQuADV1Corpus()

train_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode='train',
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/korquad-v1/cached_train_BertTokenizer_maxlen-128_maxquerylen-32_docstride-64_korquad-v1_question-answering [took 17.139 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/korquad-v1/cached_train_BertTokenizer_maxlen-128_maxquerylen-32_docstride-64_korquad-v1_question-answering [took 17.139 s]


In [ ]:
train_dataset[1]

QAFeatures(input_ids=[2, 1480, 4313, 4538, 4008, 336, 4065, 4042, 3231, 23243, 19143, 13985, 12449, 9194, 4105, 3385, 9411, 32, 3, 10958, 4062, 9511, 1355, 4600, 4103, 4775, 5602, 10770, 4180, 26732, 3231, 23243, 4104, 4042, 2015, 4012, 4113, 9198, 8763, 8129, 17, 10384, 23008, 7971, 2170, 4408, 4011, 4147, 4042, 17015, 4091, 23008, 21056, 4165, 323, 4175, 4158, 11413, 2273, 4043, 7966, 1543, 4775, 4170, 4042, 341, 4573, 4771, 28, 4566, 4027, 10599, 18907, 208, 9504, 24835, 15, 11060, 2451, 4780, 4032, 18548, 4113, 3231, 23243, 4104, 4042, 1843, 4771, 7965, 28987, 4153, 2451, 15489, 4113, 13928, 17283, 575, 4261, 26783, 8114, 8852, 9107, 4082, 28498, 8131, 17, 8225, 4042, 1114, 4281, 4194, 17138, 4042, 9961, 8222, 14041, 10892, 4113, 2524, 4443, 8032, 12710, 21602, 18625, 24569, 4136, 3], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

- train_dataset.input_ids: 원본 문장을 토큰화한 결과
- train_dataset.attention_mask: 해당 위치의 토큰이 패딩(0)인지 아닌지(1)
- train_dataset.token_type_ids:[CSL]질문[SEP]에 해당하는 세그먼트는0 지문에해당하는 두번째 세그먼트는1, 나머지 패딩에 속하는 세그먼트는 0

In [ ]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=32,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=4,
)

- sampler : 샘플링 방식을 정의
- collate_fn: 인스턴스를 배치로 만드는 역할

## 평가용 데이터 로더 구축

In [ ]:
# 평가용 데이터 로더 구축
from torch.utils.data import SequentialSampler

val_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode='val',
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=32,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=4,
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/korquad-v1/cached_val_BertTokenizer_maxlen-128_maxquerylen-32_docstride-64_korquad-v1_question-answering [took 1.255 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/korquad-v1/cached_val_BertTokenizer_maxlen-128_maxquerylen-32_docstride-64_korquad-v1_question-answering [took 1.255 s]


https://ratsgo.github.io/nlpbook/docs/qa/detail/
다른 데이터셋을 고치려면 qa 들어가서 다른모델 사용하기를 참고

## 모델 불러오기

### 모델 초기화
- BertForQuestionAnswering은 프리트레인을 마친 BERT 모델

In [ ]:
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    'beomi/kcbert-base',
)
model = BertForSequenceClassification.from_pretrained(
    'beomi/kcbert-base',
    config=pretrained_model_config,
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

## 모델 학습시키기
  - 파이토치 라이트닝이 제공하는 라이트닝 모듈을 상속받아 태스크를 정의/

In [ ]:
from ratsnlp.nlpbook.qa import QATask
task = QATask(model, args)

In [ ]:
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader
)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/optimizer.py:381: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  f"Found unsupported keys in the optimizer configuration: {set(extra_keys)}", category=RuntimeWarning
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.680   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

TypeError: ignored

In [ ]:
!pip install -U --no-cache-dir gdown --pre
print("Downloading...")
!gdown {"1-TtX8FfhVzVCQJPFUyeue_vDLtzcDfLI"} -O /content/drive/MyDrive/nlpbook/checkpoint-qa/epoch=0-val_loss=0.45.ckpt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1-TtX8FfhVzVCQJPFUyeue_vDLtzcDfLI 



In [ ]:
# !mkdir /content/drive/MytDrive/nlpbook
# !mkdir /content/drive/MytDrive/nlpbook/checkpoint-qa

mkdir: cannot create directory ‘/content/drive/MytDrive/nlpbook’: No such file or directory
mkdir: cannot create directory ‘/content/drive/MytDrive/nlpbook/checkpoint-qa’: No such file or directory


In [ ]:
from ratsnlp.nlpbook.qa import QADeployArguments
args = QADeployArguments(
    pretrained_model_name='beomi/kcbert-base',
    downstream_model_dir='/content/drive/MyDrive/nlpbook/checkpoint-qa',
    max_seq_length=128,
    max_query_length=32,
)

downstream_model_checkpoint_fpath: /content/drive/MyDrive/nlpbook/checkpoint-qa/epoch=0-val_loss=0.45.ckpt


### 체크포인트 로드

In [ ]:

import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device('cpu')
)

### BERT 설정, 모델 초기화 진행

In [ ]:
# BERT 설정 로드
from transformers import BertConfig
from transformers import BertForSequenceClassification

pretrained_model_config =BertConfig.from_pretrained(
    'beomi/kcbert-base',
)
model = BertForSequenceClassification(pretrained_model_config)

In [ ]:
# 체크포인트 읽기
model.load_state_dict({k.replace('model.',""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

RuntimeError: ignored

In [ ]:
# INFERENCE
def inference_fn(question, context):
  if question and context:
      truncated_query = tokenizer.encode(
        question,
        add_special_tokens=False,
        truncation=True,
        max_length=32
    )
      inputs = tokenizer.encode_plus(
        text=truncated_query,
        text_pair=context,
        max_length=128,
        padding='max_length',
        truncation='only_second',
        return_token_type_ids=True,
    )
      with torch.no_grad():
        outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})
        start_pred = outputs.start_logits.argmax(dim=-1).item()
        end_pred = outputs.end_logits.argmax(dim=-1).item()
        pred_text = tokenizer.decode(inputs['input_ids']\
                                    [start_pred:end_pred+1])
  else:
     pred_text = ""


In [ ]:
!mkdir /root/.ngrok2 && echo "authtoken: 2Ey1spuRdQ0sCjMXTEvvtR08odp_5avYiwqBsCqUmafHMREBt" > /root/.ngrok2/ngrok.yml


In [ ]:
from ratsnlp.nlpbook.classification import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://81fb-104-199-127-55.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


In [ ]:
inference_fn('개통된 년도는 언제인가?','한강대교(漢江大橋)는 서울특별시 용산구 이촌동에 있는 용산구 한강로3가와 동작구 본동 사이를 잇는 총연장 1,005m의 길이의 교량(다리)이다. 한강에 놓인 최초의 도로 교량으로, 제1한강교라고 불렸다. 1917년 개통된 뒤 몇 차례의 수난을 거쳐 지금에 이른다.')


In [ ]:
contxt = '파이썬은 1991년 네덜란드계 프로그래머인 귀도 반 로섬이 발표한 고급 프로그래밍 언어로, 플랫폼에 독립적이며 인터프리터식, 객체지향적, 동적 타이핑(dynamically typed) 대화형 언어이다. 파이썬이라는 이름은 귀도가 좋아하는 코미디 〈Monty Python\'s Flying Circus〉에서 따온 것이다. '
inference_fn('파이썬을 발표한 사람은?', contxt)
